# 函數庫
1. 獨立樣本檢定函數，自動化判別，並將顯著結果輸出。

In [1]:
import scipy.stats as stats
import pandas, numpy, re, os

def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

# 執行 Shapiro-Wilk_test、(常態性檢定)
def Shapiro_Wilk_test(df) :
    p_value = stats.shapiro(df).pvalue
    return p_value


# 執行變異數同質性檢定
def levene_test(df_1, df_2) :
    p_value = stats.levene(df_1, df_2).pvalue
    return p_value

# 執行獨立樣本 T 檢定
def ttest_ind_test(df_1, df_2) :
    p_value = stats.ttest_ind(df_1, df_2).pvalue
    return p_value

In [2]:
df = pandas.read_csv(os.path.join(os.getcwd(), "database_clean", "database_clean.csv"))

# 統計檢定部分
1. 相同職業在台北市與其餘縣市，在薪資水平之獨力樣本T檢定

In [3]:
# 判斷 輸出資料夾是否存在
output_folder_path = os_path(["picture", "Independent_Sample_t_test", "salary", "county", "Mix"])
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
else:
    pass

In [6]:
df.columns

Index(['Profession_classification', 'company_name', 'work_name',
       'requirements', '104_url', 'area', 'education', 'industry', 'seniority',
       'number_of_applicants', 'salary', 'company_size', 'area_district',
       'number_of_applicants_level', 'education_specialist', 'education_none',
       'education_University', 'education_high_school',
       'education_low_high_school', 'education_phd', 'education_master',
       'month_salary_classfication', 'year_salary_classfication',
       'time_salary_classfication', 'element_salary_classfication',
       'max_salary', 'mix_salary', 'company_area',
       'Profession_classification_chinese'],
      dtype='object')

## 常態性檢驗

In [7]:
# 先進行常態性檢定。
Shapiro_Wilk_test_list = []
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) :  
    for area in list(dict.fromkeys(df["area"])) : 
        # 個別做常態性檢定
        try : 
            Shapiro_Wilk_test_p = Shapiro_Wilk_test(df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification)& ~(df["mix_salary"].isnull()) & (df["month_salary_classfication"] == 1)]["mix_salary"])
            if Shapiro_Wilk_test_p <= 0.05 : 
                Shapiro_Wilk_test_list.append({"縣市" : area ,"職業類型" : Profession_classification, "是否通過_Shapiro_Wilk" : "pass"})
            else : 
                Shapiro_Wilk_test_list.append({"縣市" : area ,"職業類型" : Profession_classification, "是否通過_Shapiro_Wilk" : "none"})
        except : 
            Shapiro_Wilk_test_list.append({"縣市" : area ,"職業類型" : Profession_classification, "是否通過_Shapiro_Wilk" : "none"})

Shapiro_Wilk_test_df = pandas.DataFrame(Shapiro_Wilk_test_list)
len(Shapiro_Wilk_test_df[Shapiro_Wilk_test_df["是否通過_Shapiro_Wilk"]=="pass"])/len(Shapiro_Wilk_test_df) # 通過率 0.984
# 有意思的是，幾乎都沒有辦法通過呢~


c:\Users\Sigolon\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


0.9841269841269841

## 變異數同質性檢定


In [8]:
levene_test_list = []
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) :  
    for area in list(dict.fromkeys(df["area"])) : 
        if area == "台北市" : 
            continue

        df_none_Taipai = df[(df["area"] == area) & (df["Profession_classification_chinese"] == Profession_classification) & ~(df["mix_salary"].isnull()) & (df["month_salary_classfication"] == 1)]["mix_salary"]
        df_Taipai = df[(df["area"] == "台北市") & (df["Profession_classification_chinese"] == Profession_classification) & ~(df["mix_salary"].isnull()) & (df["month_salary_classfication"] == 1)]["mix_salary"]
        
        # 變異數同質性檢定
        try : 
            levene_test_p = stats.levene(df_none_Taipai, df_Taipai).pvalue
            if levene_test_p <= 0.05 : 
                levene_test_list.append({"縣市" : area ,"職業類型" : Profession_classification, "是否通過_levene" : "pass"})
            else : 
                levene_test_list.append({"縣市" : area ,"職業類型" : Profession_classification, "是否通過_levene" : "none"})
        except : 
            levene_test_list.append({"縣市" : area ,"職業類型" : Profession_classification, "是否通過_levene" : "none"})

levene_test_df = pandas.DataFrame(levene_test_list)
len(levene_test_df[levene_test_df["是否通過_levene"]=="pass"])/len(levene_test_df) # 通過率 0.62

c:\Users\Sigolon\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Sigolon\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.6222222222222222

In [9]:
levene_test_df["是否通過_Shapiro_Wilk"] = Shapiro_Wilk_test_df["是否通過_Shapiro_Wilk"]


In [15]:
# 使用np.where根据条件生成新的列
levene_test_df["能否進行檢定"] = numpy.where((levene_test_df['是否通過_levene'] == "pass") & (levene_test_df['是否通過_Shapiro_Wilk'] == "pass"), "pass", "none")
len(levene_test_df[levene_test_df["能否進行檢定"] == "pass"])/len(levene_test_df)

0.6138888888888889

In [16]:
levene_test_df

,縣市,職業類型,是否通過_levene,是否通過_Shapiro_Wilk,能否進行檢定
0,新北市,經營、人資類,pass,pass,pass
1,基隆市,經營、人資類,none,pass,none
2,桃園市,經營、人資類,none,pass,none
3,台南市,經營、人資類,pass,pass,pass
4,宜蘭縣,經營、人資類,pass,pass,pass
...,...,...,...,...,...
355,澎湖縣,其他類,none,pass,none
356,苗栗縣,其他類,pass,pass,pass
357,嘉義縣,其他類,pass,pass,pass
358,嘉義市,其他類,pass,pass,pass


In [17]:
levene_test_df.to_csv(os_path(["檢定資料", "同質性檢定.csv"]), index= False)

In [ ]:
levene_test_df["縣市"].unique()

array(['新北市', '基隆市', '桃園市', '台南市', '宜蘭縣', '花蓮縣', '彰化縣', '雲林縣', '金門縣',
       '新竹縣', '台中市', '高雄市', '南投縣', '屏東縣', '新竹市', '澎湖縣', '苗栗縣', '嘉義縣',
       '嘉義市', '台東縣'], dtype=object)

# 本段落小結 : 
透過這些圖表，試圖回答以下問題 : 
1. 不同學門之學生，畢業以後應該往何處進行物理性質的流動，才能夠擁有更優渥的薪水又或者說「機會」?
2. 如果要進行流動的話，又該具備何種技能，才能夠確保自己擁有流動的資本 ? 
3. 這些技能是否具備地區上的同質性呢 ? 如果不是，那除了薪水以外，我們確實應該也要考慮，何地所欠缺的人力資本，與自身更有匹配度了 ?